# Linear Regression for Business Statistics <a id="heading"></a>

1. [**Regression Analysis: An Introduction**](#week-1-rai)
2. [**Regression Analysis: Hypothesis Testing and Goodness of Fit**](#week-2-rahtagof)
3. [**Regression Analysis: Dummy Variables, Multicollinearity**](#week-3-radvm)
4. [**Regression Analysis: Various Extensions**](#week-4-rave)
---

## Regression Analysis: An Introduction <a id="week-1-rai"></a>

[*Back to the heading*](#heading)

**What is Linear Regression?**

**Linear regression** attempts to fit a linear relation between a variable of interest and a set of variables that may be related to the variable of interest.

There are two main types of Linear Regression:

* Simple Regression - a regression with only one explanatory or X variable
* Multiple Regression - a regression with two or more explanatory or X variables

Overview of Regression:

1. Modeling - developing a regression model
2. Estimation - using software to estimate the model
3. Inference - interpreting the estimated regression model
4. Prediction - making predictions about the variable of interest

---

Example:

There is a Sales manager of a toys retail company which sells various kinds of toys in the local market. This sales manager needs to make some kind of projections about the number of monthly units that the retail company will be able to sell of this particular toy in the coming half year. In the past she has been making such projections based on her gut feelingand now wishes to be a little more scientific about the whole process.

Based on her experience, the manager figures out that the **monthly unit sales** depend on three important variables, the **price** at which the toy is sold, the **monthly amount that the company spends on advertising** the toy and the **monthly amount spent on promotions** for the toy.

The final formula would be:

$ Sales = \beta_0 + \beta_1 Price + \beta_2 AdExp + \beta_3 PromExp $

Where:

* Sales - variable of interest / Y / Dependent / Response / Regressed / L.H.S.
* Price, AdExp, PromExp - Explanatory / X / Independent / Covariate / Regressor / R.H.S. variable
* $\beta_x$ - Coefficient Parameters (calculated)

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import t
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from math import sqrt, exp, log

In [2]:
toys_data = pd.read_excel("Excel files/Toy-sales.xlsx", "Sheet1", index_col=None, na_values=["NA"])

In [3]:
toys_data.head()

,Month,Unit Sales,Price ($),Adexp ('000$),Promexp ('000$)
0,1,73959,8.75,50.04,61.13
1,2,71544,8.99,50.74,60.19
2,3,78587,7.50,50.14,59.16
3,4,80364,7.25,50.27,60.38
4,5,78771,7.40,51.25,59.71


In [4]:
X = toys_data[["Price ($)", "Adexp (\'000$)", "Promexp (\'000$)"]]
Y = toys_data["Unit Sales"]

# via sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)  # works the same way
print("Regression via sklearn:\nCoefficients:", regr.coef_, "\n")
price, adexp, promexp = 8.1, 50, 60
sales_predict = regr.predict([[price, adexp, promexp]])
print(f"Predicted sales with Price {price}, AdExp {adexp}, PromExp {promexp} is: {sales_predict}")
print(f"R^2: {regr.score(X, Y)}, Intercept: {regr.intercept_}")
print("\n", "_" * 78, "\n\nRegression via statsmodels\n", sep="")

# via statsmodels
X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print_model = model.summary()  # alpha=0.05 by default
print(print_model)

Regression via sklearn:
Coefficients: [-5055.26986592   648.61214026  1802.61095612] 

Predicted sales with Price 8.1, AdExp 50, PromExp 60 is: [74542.74554463]
R^2: 0.8588446525398427, Intercept: -25096.832921870096

______________________________________________________________________________

Regression via statsmodels

                            OLS Regression Results                            
Dep. Variable:             Unit Sales   R-squared:                       0.859
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     40.56
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           1.08e-08
Time:                        21:20:23   Log-Likelihood:                -203.48
No. Observations:                  24   AIC:                             415.0
Df Residuals:                      20   BIC:                             419.7
Df Model:                           3     

$R^2$ here is the proportion of variation in the Y variable explained by the regression model. The closer $R^2$ to 1 the better the fit.

**Regression is a process that has errors**
* Residuals and Errors
* Residuals $= Y^{actual} - Y^{predicted}$
* Errors are typically distributed equally above and below the regression line
* R-square: A “goodness of fit” measure

**Why do we have errors in the regression model?**
* Omitted variables
* Functional relationship between the Y and X variables
* The theory of regression analysis is based on certain assumptions about these errors

**NB:** $\beta_i$ is actually unknown (just like $\mu$). What we estimate are $b_i$, although analysts quite often mix these notation. $b_i$'s have normal distribution with $\beta_i$ as their mean. $b_i$'s can be considered as random variables: 

$$b_i \sim Normal(\beta_i, \text{some std})$$

Some important results that enable us to test stability and precision of coefficients and conduct hypothesis testing in a regression:

$$ \frac{b_i - \beta_i}{s_{b_i}} \sim t_{n-k-1} $$

Where:
* $i$ - number of the parameter
* $n$ - number of observations
* $k$ - number of "X" variables
* $n-k-1$ - residual degrees of freedom
* $s_{b_i}$ - the standard error of $b_i$

### Quiz 1

In [5]:
quiz_data = pd.read_excel("Excel files/1. Grocery Store Sales.xlsx", "Sheet1", index_col=None, na_values=["NA"])

In [6]:
quiz_data.head()

,Store,Sales per Square Foot ($),Size of Store (in Sq. Ft.),Advertising Dollars,Number of Products Offered in Store
0,1,837,64796,22000,32920
1,2,748,74179,58000,25034
2,3,744,70298,58000,23989
3,4,853,63367,56000,31095
4,5,839,74412,67000,35055


1.

Download Grocery Store Sales, which provides data in the following categories: Sales per Square Foot, Size of Store (in Square Feet), Advertising Dollars (in thousands), and Number of Products Offered in Store, from a sample size of 70 grocery stores.

We want to see how changes in our independent variables affect Sales per Square Foot.

Please run one multiple regression including all independent variables to estimate the coefficients for each of our independent variables. 

What is the coefficient for Size of Store? Please round to three decimal places.

In [7]:
X = quiz_data[["Size of Store (in Sq. Ft.)", "Advertising Dollars", "Number of Products Offered in Store"]]
Y = quiz_data["Sales per Square Foot ($)"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print_model = model.summary()
print(print_model)

                                OLS Regression Results                               
Dep. Variable:     Sales per Square Foot ($)   R-squared:                       0.383
Model:                                   OLS   Adj. R-squared:                  0.355
Method:                        Least Squares   F-statistic:                     13.65
Date:                       Tue, 20 Apr 2021   Prob (F-statistic):           5.00e-07
Time:                               21:20:23   Log-Likelihood:                -373.65
No. Observations:                         70   AIC:                             755.3
Df Residuals:                             66   BIC:                             764.3
Df Model:                                  3                                         
Covariance Type:                   nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

5.

What would be the expected Sales per Square Foot if the Size of Store was 60,000 square feet, they spent \\$70,000 in Advertising Dollars, and offered 30,000 products (in \$) ? Please round to two decimal places. 

In [8]:
round(model.predict([1, 60000, 70000, 30000])[0], 2)  # 1 is added constant

893.43

---
## Regression Analysis: Hypothesis Testing and Goodness of Fit <a id="week-2-rahtagof"></a>

[*Back to the heading*](#heading)

The estimate value ($b$) doesn't mean that it equals only to this value. The analyst can use hypothesis testing (in different ways) to confirm their hypothesis to check whether another value can be used instead of the estimate.

To test the hypothesis you could use:
* The t-cutoff approach: t-statistic ($\frac{b_i-\beta_i}{s_{b_i}}$) should lie outside of the rejection region
* The p-value approach: the null hypothesis is rejected when $\text{p-value} < \alpha$
* The confidence interval approach: the easiest one (the most used one as well), the hypothesis value should lie within the confidence interval

Let's see the last model from the quiz to look at some other data.

In [9]:
print(print_model)

                                OLS Regression Results                               
Dep. Variable:     Sales per Square Foot ($)   R-squared:                       0.383
Model:                                   OLS   Adj. R-squared:                  0.355
Method:                        Least Squares   F-statistic:                     13.65
Date:                       Tue, 20 Apr 2021   Prob (F-statistic):           5.00e-07
Time:                               21:20:23   Log-Likelihood:                -373.65
No. Observations:                         70   AIC:                             755.3
Df Residuals:                             66   BIC:                             764.3
Df Model:                                  3                                         
Covariance Type:                   nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

Here the p-values of the coefficients mean the p-values of the hypothesis value equal to 0. Thus we can assume that if the p-value of the coefficient is less than $\alpha$ then this coefficient is meaningful.

Another important value here is `R-squared`. It represents the "goodness" (or goodness-of-fit) of the regression, which in turn means the proportion of variation in the Y variable explained by the regression model. The closer $R^2$ to 1 the better the fit. $R^2$ only goes up when you add additional X variables (even random variables), but Adjusted R-squared fixes this issue - it goes down when you add unmeaningful variables.

Sometimes we have categorical (strings) data in X variables, for example region, profession, eye color, etc. To deal with them we'll need to create so-called dummy variables. Each category has a distinct column with binary (1 or 0) values, in case of using a constant (intercept) you'll need to drop one of those columns. 

For example you have "brown eyes", "grey eyes", "yellow eyes" as categories. Thus you'll need only two coulmns like "BE" (for brown eyes) and "GE" (for grey eyes). If both of them are 0 - then the eye color is yellow. See an example of using of the dummy variables in the quiz below.

### Quiz 2

In [10]:
quiz_data = pd.read_excel("Excel files/2. Final Exam Scores.xlsx", "Sheet1", index_col=None, na_values=["NA"])

In [11]:
quiz_data.head()

,Final Exam Score,Attended Review Session,Mid-Term Score,Homework Score
0,75,No,69,96
1,82,No,87,80
2,88,Yes,71,99
3,83,No,66,79
4,72,Yes,61,70


1.

Please run a multiple regression with 'Final Exam Score' as the dependent variable and the remaining variables as independent variables. Remember, for categorical variable(s), you will need to create dummy or indicator variable(s). 

Which of the following variables is not statistically significant? Assume an alpha level of .05.

>**Note**: `pd.get_dummies` (returns a dataframe with dummy variables for the category columns without the category columns) has bool `drop_first` parameter.

In [12]:
X = pd.get_dummies(quiz_data, drop_first=True)[["Attended Review Session_Yes", "Mid-Term Score", "Homework Score"]]
Y = quiz_data["Final Exam Score"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:       Final Exam Score   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     6.850
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           0.000907
Time:                        21:20:23   Log-Likelihood:                -132.10
No. Observations:                  40   AIC:                             272.2
Df Residuals:                      36   BIC:                             279.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

2.

If a student received scores of 0 on both the Mid-Term and Homework, and did not attend the Review Session, what would one predict his or her score on the final exam to be? Please round to the nearest whole number. 

In [13]:
X.head()

,const,Attended Review Session_Yes,Mid-Term Score,Homework Score
0,1.0,0,69,96
1,1.0,0,87,80
2,1.0,1,71,99
3,1.0,0,66,79
4,1.0,1,61,70


In [14]:
round(model.predict([1, 0, 0, 0])[0])  # 1 is added constant

52

3.

There is a belief among students that if they attend the Review Session, it will increase their Final Exam Scores by 10 points. You need to evaluate this belief by setting up an appropriate hypothesis test.

First, please calculate the t-statistic for this hypothesis test. Please round to two decimal points. 

In [15]:
t_statistic = (model.params["Attended Review Session_Yes"] - 10) / model.bse["Attended Review Session_Yes"]
print(round(t_statistic, 2))

-1.77


4.

Please calculate the t-cutoff for this hypothesis testing; round the answer to two decimal points. Assume α = .05.

What is the absolute value of the t-cutoff?

In [16]:
t_cutoff = abs(t.ppf(0.05/2, model.df_resid))
print(round(t_cutoff, 2))

2.03


---
## Regression Analysis: ~Dummy Variables,~ Multicollinearity <a id="week-3-radvm"></a>

[*Back to the heading*](#heading)

Let's talk about Multicollinearity.

In [17]:
cars_data = pd.read_excel("Excel files/Cars.xlsx", "Sheet1", index_col=None, na_values=["NA"])

In [18]:
cars_data.head()

,Car_Model,MPG,Displacement,Cylinders
0,1,16.9,350,8
1,2,15.5,351,8
2,3,19.2,267,8
3,4,18.5,360,8
4,5,30.0,98,4


Let's make a linear regression to see the impact of displacement and cylinders on miles per galon of a car.

In [19]:
X = cars_data[["Displacement", "Cylinders"]]
Y = cars_data["MPG"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.636
Method:                 Least Squares   F-statistic:                     33.36
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           7.79e-09
Time:                        21:20:23   Log-Likelihood:                -104.55
No. Observations:                  38   AIC:                             215.1
Df Residuals:                      35   BIC:                             220.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           40.6163      3.187     12.744   

Here we can see that the p-values of both X variables are higher than $\alpha$. But at the same time the R-squared seems just fine. When such situations happen, there's a high chance of **multicollinearity** in your data.

Let's check linear regressions with just one X variable.

In [20]:
print("MPG-Displacement linear regression:\n")
X = cars_data["Displacement"]
Y = cars_data["MPG"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print_model = model.summary()
print(print_model)
print("\n", "_" * 78, "\n", sep="")

print("MPG-Cylinders linear regression:\n")
X = cars_data["Cylinders"]
Y = cars_data["MPG"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print_model = model.summary()
print(print_model)

MPG-Displacement linear regression:

                            OLS Regression Results                            
Dep. Variable:                    MPG   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     58.21
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           4.99e-09
Time:                        21:20:23   Log-Likelihood:                -106.54
No. Observations:                  38   AIC:                             217.1
Df Residuals:                      36   BIC:                             220.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const      

Now both X variables are "significant" as their p-values are lower than $\alpha$. This definetely shows us that there's  multicollinearity problem.

**Multicollinearity**
>* Occurs when two or more X variables are highly correlated
>* Variation in Y cannot be apportioned individually across X variables
>* Model fits may be reasonable and prediction may be ok
>* Interpretation of individual X variable impact will be suspect

Our car model may predict reasonably. However, individual impact of Displacementand Cylinderscannot be discerned.

**Detecting Multicollinearity**
>* Some software produce collinearity statistics such as VIF (Variance Inflation Factor).
>* We can look at correlations across X variables.
>* `statsmodel` notes the condition number
>* Rule of thumb: if there's correlation >= 0.89 then there's multicollinearity
>* Another Rule of thumb: a VIF greater than 10 (some consider 4 or 6) indicates presence of multicollinearity

In [21]:
cars_data[["Displacement", "Cylinders"]].corr()

,Displacement,Cylinders
Displacement,1.000000,0.940281
Cylinders,0.940281,1.000000


In [22]:
variables = sm.add_constant(cars_data[["Displacement", "Cylinders"]]).values
vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
print(vif[1:]) # dropped constant column

[8.630267037760873, 8.630267037760873]


**Correcting Multicollinearity**
* Do you need to correct for collinearity?
    + May not be a problem if using regression only to predict
    + Collinearity is problematic when interpreting coefficient impacts
* To remove multicollinearity you may drop variables causing high correlation

## Quiz 3

In [23]:
quiz_data = pd.read_excel("Excel files/3. Sales By Territory.xlsx", "Sales Revenue", index_col=None, na_values=["NA"])

In [24]:
quiz_data.head()

,Sales Revenue,Territory,Quantity of Orders,Number of Sales Calls
0,21863,East,162,17
1,53633,East,224,22
2,35530,South,271,29
3,23037,West,103,13
4,48444,North,196,23


1.

Please run a multiple regression with Sales Revenue as your dependent variable to estimate the coefficients for each of the independent variables; remember, for categorical variables, you will need to create dummy or indicator variables. Please use “West” Territory as your reference variable, and assume an alpha of .05. 

Which of the following variables are statistically significant according to the p-values?

In [25]:
X = pd.get_dummies(quiz_data).drop(["Sales Revenue", "Territory_West"], axis=1)
Y = quiz_data["Sales Revenue"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:          Sales Revenue   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     10.89
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           3.52e-06
Time:                        21:20:24   Log-Likelihood:                -401.53
No. Observations:                  38   AIC:                             815.1
Df Residuals:                      32   BIC:                             824.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -2815.49

In [26]:
model.params

const                    -2815.497554
Quantity of Orders         115.025757
Number of Sales Calls     1280.295335
Territory_East           -3254.290247
Territory_North          -1296.587003
Territory_South         -12240.753287
dtype: float64

5.

Please estimate what Sales Revenue would be for a salesperson covering the “South” Territory, making 28 Sales Calls, and taking 200 Orders, rounding the answer to two decimal points. 

In [27]:
round(model.predict([1, 200, 28, 0, 0, 1])[0], 2)  # 1 is added constant

43797.17

9.

Within our regression, we notice that some of our independent variables have statistically insignificant p-values, yet as calculated above, the R-Square for the model is okay. This may imply some multicollinearity. Please calculate the correlation between "Number of Sales Calls” and "Quantity of Orders”, rounding to two decimal points.

In [28]:
quiz_data[["Number of Sales Calls", "Quantity of Orders"]].corr()

,Number of Sales Calls,Quantity of Orders
Number of Sales Calls,1.000000,0.763846
Quantity of Orders,0.763846,1.000000


---
## Regression Analysis: Various Extensions <a id="week-4-rave"></a>

[*Back to the heading*](#heading)

### Mean-Centering of Variables

Example:

We have a following regression model of weight:

$ \text{Weight} = \beta_0 + \beta_1Male + \beta_2Height $

If we wish to have a situation where the intercept has a meaningful interpretation in this case we'll have to make sure that all other variables being zero is meaningful in some way. Male being zero is meaningful because that implies a female olympian, however, height being zero is not meaningful. But we can make that meaningful by **mean centering** the height.

We do it the following way:
* Subtracting the mean from every value of a variable.
* It redefines the zero point for that variable.
* We could also center a variable around some value other than the mean.
* We should mean center the X variables depending on the way we want to interpret the regression

This way we'll be able to interpret the intercept as the weight of a female with an average height and the regression model will look like this: $ \text{Weight} = \beta_0 + \beta_1Male + \beta_{2}\overline{Height} $

Here's an example how we do it:

In [29]:
olymp_data = pd.read_excel("Excel files/Height-and-Weight.xlsx", "Sheet1", index_col=None, na_values=["NA"])
new_olymp_data = olymp_data.drop(["Second name", "First name", "Country/Team"], axis=1)
new_olymp_data = pd.get_dummies(new_olymp_data).drop("Gender_W", axis=1)
new_olymp_data["Height_mc"] = new_olymp_data["Height(cm)"] - new_olymp_data["Height(cm)"].mean()
new_olymp_data.head()

,Height(cm),Weight(kg),Gender_M,Height_mc
0,170,69,1,-7.451166
1,171,64,0,-6.451166
2,167,52,1,-10.451166
3,168,68,1,-9.451166
4,177,78,1,-0.451166


In [30]:
X = new_olymp_data[["Height_mc", "Gender_M"]]
Y = new_olymp_data["Weight(kg)"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Weight(kg)   R-squared:                       0.621
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                     1298.
Date:                Tue, 20 Apr 2021   Prob (F-statistic):               0.00
Time:                        21:20:24   Log-Likelihood:                -5882.6
No. Observations:                1587   AIC:                         1.177e+04
Df Residuals:                    1584   BIC:                         1.179e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         69.6267      0.407    170.868      0.0

### Confidence intervals

It's very easy with statsmodels. Look at the example below:

First, we need to recreate the model without mean centering the height values:

In [31]:
X = new_olymp_data[["Height(cm)", "Gender_M"]]
Y = new_olymp_data["Weight(kg)"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()

And now let's calculate the confidence interval for the predicted value:

In [32]:
prediction = model.predict([1, 177, 1])[0]

n = len(new_olymp_data)
alpha = 0.05
t_cutoff = abs(t.ppf(alpha/2, model.df_resid))

s = np.sqrt(model.scale)  # regression's standard error
margin_of_error = abs(t_cutoff) * s
lower = prediction - margin_of_error
upper = prediction + margin_of_error

In [33]:
print(f"Predict the weight of a male Olympian with a height of 177 cm: {round(prediction, 2)} Kg")
print(f"95% Confidence interval for the prediction: [{round(lower, 2)}, {round(upper, 2)}]")

Predict the weight of a male Olympian with a height of 177 cm: 74.72 Kg
95% Confidence interval for the prediction: [55.37, 94.06]


The confidence interval for the predicted value is a way of incorporating uncertainty in our prediction.

### Interaction Effects in a Regression Model

We can create an interaction value to have another meaningful interpretation of the coefficients and the intercept of a regression model. Let's get back to our height/weight example:

$ \text{Weight} = \beta_0 + \beta_1Male + \beta_2Height $

* Effect of Height on Weight is same across Gender
* Dummy variable Male allows for some overall differences
* No Gender difference allowed based on Height

How do we allow impact of Height based on Gender? We can use an interaction variable! This particular interaction variable is simply a multiplication of the Male and the Height variables:

$ \text{Weight} = \beta_0 + \beta_1Male + \beta_2Height + \beta_3(Male * Height) $

We can also mean center the Height variable, so the intercept have a meaningful interpretation:

$ \text{Weight} = \beta_0 + \beta_1Male + \beta_{2}\overline{Height} + \beta_3(Male * \overline{Height}) $

Thus we can interpret our coefficients like this:

* $\beta_0$ is the weight of a Female Olympian whose height is at a level equal to the average height observed in the data
* $\beta_1$ the additional weight of a Male Olympian vis-à-vis a Female Olympian when the height is equal to the average height observed in the data
* $\beta_0 + \beta_1$ is the total weight of a Male Olympian whose height is equal to the average height observed in the data
* $\beta_2$ is the impact of one centimeter increase in height on the weight of Female Olympians
* $\beta_3$ is the additional impact of Height on weight, for Male Olympians
* $\beta_2 + \beta_3$ is the total impact of Height on weight for Male Olympians

Interaction variables are useful anytime you wish to study the impact of one explanatory variable at different levels of some other variable.

The interacting variables need not be one dummy and the other numerical. They both could be numerical variables. For example:

Salaries at workplace:
* Could interact "Years of Experience" and "Years of Education"
* Coefficient will tell us the impact of experience on salary at different levels of education
* Could also have a three way interaction by interacting 3 variables "Years of Experience", "Years of Education", "Gender"

### The Log-Log and the Semi-Log Regression Models

Why would we transform variables in a Regression?
* Regression is a linear procedure
* If the relation between Y and X variable is not linear

Various transformations that can be used, like:
* Invert one or more variables
* Take a square of a variable
* Take a square root

Bit of Trial and Error required. We also can take the Natural log transformation using:
* **Log-log Model**: $ LN(Y) = \beta_0 + \beta_1LN(X_1) + \beta_2LN(X_2) + ... $
* **Semi-log Model**: $ LN(Y) = \beta_0 + \beta_1X_1 + \beta_2X_2 + ... $

In case of the **Log-log Model**: when the natural log of $X_1$ increases by one unit, then the natural log of Y increases by 1 unit, all other variables remaining at the same level. Or in other words:
> One percentage increase in $X_1$ corresponds to a $\beta_1$ percentage increase in $Y$, all other variables remaining at the same level
> * Also known as an Elasticity interpretation. $\beta_1$ is the elasticity of $Y$ with respect to $X_1$

In case of the **Semi-log Model**: 
> One unit increase in $X_1$ corresponds to a $100 * \beta_1$ percentage increase in $Y$, all other variables remaining at the same level
> * Also known as Growth Rate interpretation

Two main reasons for taking a natural log transformation:
1. Improve Linearity
2. Have "Betas" be interpreted as Elasticities or Growth Rates

Also note that a regression model can be Log-log and Semi-log at the same time (taking the Natural log only of several variables).

### Quiz 4

In [34]:
quiz_data = pd.read_excel("Excel files/4. realestate.xlsx", "Sheet1", index_col=None, na_values=["NA"])
# quiz_data = pd.read_excel("Excel files/4. Majors.xlsx", "Sheet1", index_col=None, na_values=["NA"])

In [35]:
quiz_data.head()

,PRICE ($),SQFT,BED,BATH,FLOORS,DIST (km)
0,650000.0,1001,2,1,4,1.909
1,380000.0,735,1,1,13,0.128
2,602500.0,790,1,1,4,2.817
3,180000.0,413,0,1,3,2.360
4,320000.0,718,1,1,13,0.128


The data contains information about apartment prices and characteristics for a sought after area in a large metropolitan city in the USA. The data include sale price (PRICE) in $, floor area (SQFT) in square feet, number of bedrooms (BED), number of bathrooms (BATH), number of floors in the building (FLOORS), and distance from a centrally located city park (DIST) in meters.

1.

You need to establish a relationship between PRICE and these other characteristics. Specifically, estimate the following regression model,

$LN(PRICE) = \beta_0 + \beta_1LN(SQFT) + \beta_2BED + \beta_3BATH + \beta_4FLOORS + \beta_5DIST$

Notice that in the regression you need to take a log transformation of PRICE and SQFT variables. Report the estimated value of $\beta_4$, round the answer to four decimal digits.

In [36]:
quiz_data["L_PRICE"] = np.log(quiz_data["PRICE ($)"])
quiz_data["L_SQFT"] = np.log(quiz_data["SQFT"])

X = quiz_data[["L_SQFT", "BED", "BATH", "FLOORS", "DIST (km)"]]
Y = quiz_data["L_PRICE"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                L_PRICE   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     524.8
Date:                Tue, 20 Apr 2021   Prob (F-statistic):          3.20e-210
Time:                        21:20:24   Log-Likelihood:               -0.75713
No. Observations:                 574   AIC:                             13.51
Df Residuals:                     568   BIC:                             39.63
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.2121      0.258     24.106      0.0

In [37]:
print(round(model.params[4], 4))

-0.0098


4.

Using the estimated regression model, predict the price in dollars of an apartment that is 1000 sqft in size, has 2 Bedrooms, 2 Bathrooms, is in a building with 8 Floors and is 1.2 Km from the City Park. Round your answer to a whole number, input the answer without any "$" or "," sign.

In [38]:
log_prediction = model.predict([1, log(1000), 2, 2, 8, 1.2])[0]
print("Log prediction:", log_prediction)
prediction = exp(log_prediction)
print("$ prediction:", round(prediction))

Log prediction: 13.296935793641374
$ prediction: 595368


5.

Calculate a 95% confidence interval for your predicted price from Question 4. Report the lower limit of the confidence interval (in dollars), round your answer to a whole number.

In [39]:
n = len(quiz_data)
alpha = 0.05
t_cutoff = abs(t.ppf(alpha/2, model.df_resid))

s = np.sqrt(model.scale)  # regression's standard error
margin_of_error = t_cutoff * s
lower = log_prediction - margin_of_error
upper = log_prediction + margin_of_error
print(f"95% Confidence interval for the prediction: [{round(exp(lower))}, {round(exp(upper))}]")

95% Confidence interval for the prediction: [368994, 960622]


6.

Data for Questions 6 through 11 is contained in the file Majors.xlsx.

The data contains information about the starting salary of a sample of 50 undergraduate students at a Business school. The data consists of the starting salary (SALARY) in dollars, the field of study of the student (MAJOR), the field of study is either 'Finance' or 'International Business'. Finally, the variable UGPA is undergraduate Grade Point Average of the student.

Estimate a regression model linking starting salary to the field of study and UGPA as follows,

SALARY = $\beta_0$ + $\beta_1IB$ + $\beta_2UGPA$

In the above regression, $IB$ is a dummy variable which takes a value =1 when the MAJOR is IB, otherwise it takes a value 0. 

Report the estimated value of $\beta_1$, round the answer to a whole number.

In [40]:
quiz_data = pd.read_excel("Excel files/4. Majors.xlsx", "Sheet1", index_col=None, na_values=["NA"])

In [41]:
quiz_data.head()

,SALARY,MAJOR,UGPA
0,64057,FINANCE,3.6
1,55111,FINANCE,2.6
2,58018,IB,2.7
3,54426,IB,2.2
4,69205,FINANCE,3.2


In [42]:
quiz_data = pd.get_dummies(quiz_data, drop_first=True).rename(columns = {"MAJOR_IB": "IB"})

X = quiz_data[["IB", "UGPA"]]
Y = quiz_data["SALARY"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print(model.summary())
print("Beta 1 is:", round(model.params[1]))

                            OLS Regression Results                            
Dep. Variable:                 SALARY   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     43.21
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           2.25e-11
Time:                        21:20:24   Log-Likelihood:                -476.89
No. Observations:                  50   AIC:                             959.8
Df Residuals:                      47   BIC:                             965.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        4.73e+04   4152.040     11.392      0.0

7.

Now, mean center the UGPA variable. That is, subtract the mean value of UGPA from all the data points. Denote this mean centered variable as [UGPA].

Run a regression as follows,

$SALARY = \beta_0 + \beta_1IB + \beta_2[UGPA]$

Round the estimated value of $\beta_0$ to a whole number and interpret it. Please mark all that apply.

In [43]:
quiz_data.head()

,SALARY,UGPA,IB
0,64057,3.6,0
1,55111,2.6,0
2,58018,2.7,1
3,54426,2.2,1
4,69205,3.2,0


In [44]:
quiz_data["[UGPA]"] = quiz_data["UGPA"] - quiz_data["UGPA"].mean()

X = quiz_data[["IB", "[UGPA]"]]
Y = quiz_data["SALARY"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print(model.summary())
print("Beta 0 is:", round(model.params[0]))

                            OLS Regression Results                            
Dep. Variable:                 SALARY   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     43.21
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           2.25e-11
Time:                        21:20:24   Log-Likelihood:                -476.89
No. Observations:                  50   AIC:                             959.8
Df Residuals:                      47   BIC:                             965.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.063e+04    821.531     73.801      0.0

8.

Based on the regression carried out in Question 7, how much less salary (in dollars) does a IB Major get as compared to a FINANCE Major, when they have the same UGPAs. Round your answer to a whole number. Input the answer without any "$" or "," sign.

In [45]:
print(round(model.params[1]))

-5145


9.

There is a belief among students that higher UGPA is more important in terms of impacting starting salary for IB undergraduates as compared to FINANCE undergraduates.

You can empirically check for this belief by introducing an interaction variable in your regression model constructed in Question 7 and then checking the estimated coefficient for that variable.

To introduce the interaction variable which variables would you interact.

In [46]:
quiz_data.head()

,SALARY,UGPA,IB,[UGPA]
0,64057,3.6,0,0.692
1,55111,2.6,0,-0.308
2,58018,2.7,1,-0.208
3,54426,2.2,1,-0.708
4,69205,3.2,0,0.292


In [47]:
quiz_data["IB * [UGPA]"] = quiz_data["[UGPA]"] * quiz_data["IB"]

X = quiz_data[["IB", "[UGPA]", "IB * [UGPA]"]]
Y = quiz_data["SALARY"]

X = sm.add_constant(X)
model = sm.OLS(Y, X, hasconst=True).fit()
print(model.summary())
print("Beta 3 is:", round(model.params[3]))

                            OLS Regression Results                            
Dep. Variable:                 SALARY   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     35.67
Date:                Tue, 20 Apr 2021   Prob (F-statistic):           4.59e-12
Time:                        21:20:24   Log-Likelihood:                -472.92
No. Observations:                  50   AIC:                             953.8
Df Residuals:                      46   BIC:                             961.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        6.142e+04    816.835     75.190      